In [111]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
import seaborn as sns
import matplotlib.pyplot as plt

In [112]:
df=pd.read_csv('model_data2.csv')

In [113]:
df.dtypes

entity_public_float                    float64
comprehensive_loss_net_of_tax          float64
assets                                 float64
cash_equivalents                       float64
comprehensive_income_net_of_tax        float64
earnings_per_share_diluted             float64
income_tax_expense_benefit             float64
liabilities_and_stockholders_equity    float64
net_from_financing_activities          float64
investments                            float64
net_from_operating_activities          float64
net_income_loss                        float64
operating_lease_liability              float64
retained_earnings                      float64
stockholders_equity                    float64
diluted_shares_outstanding             float64
cik                                      int64
ticker                                  object
company_name                            object
median_worker_pay                        int64
ceo salary                               int64
industry     

In [114]:
labels=df[['esg_score', 'controversey_score', 'environment_score', 'social_score', 'governance_score']]

In [115]:
df=df.drop(columns=['esg_score', 'controversey_score', 'cik', 'environment_score', 'social_score', 'governance_score'])

In [116]:
pd.set_option('display.float_format', lambda x: f'{x:.3f}')

In [7]:
df_num_controversey_score=df.drop(columns=[])

In [117]:
df_num_esg_scores_model = df.select_dtypes(include=np.number)
df_num_controversey_score=df_num_controversey_score.select_dtypes(include=np.number)
df_cat=df.industry


# Model to predict ESG Scores having the other individual scores as features

,entity_public_float,comprehensive_loss_net_of_tax,assets,cash_equivalents,comprehensive_income_net_of_tax,earnings_per_share_diluted,income_tax_expense_benefit,liabilities_and_stockholders_equity,net_from_financing_activities,investments,...,retained_earnings,stockholders_equity,diluted_shares_outstanding,median_worker_pay,ceo salary,Beta,revenue,gross_profit,total_debt,market_cap
0,26600000000.000,-280000000.000,10919000000.000,-431000000.000,1189000000.000,1.190,58000000.000,10919000000.000,-17000000.000,-104000000.000,...,541000000.000,5609000000.000,297000000.000,82890,15967631,1.000,6930000000,3720000000,3130000000,40960000000
1,8200000000.000,-3539000000.000,14756000000.000,505000000.000,930000000.000,-0.680,664000000.000,14756000000.000,-768000000.000,-495000000.000,...,-570000000.000,5076000000.000,181000000.000,102645,15960369,2.400,12450000000,2310000000,1970000000,7250000000
2,8200000000.000,-4585000000.000,64716000000.000,38000000.000,1484000000.000,0.190,59000000.000,64716000000.000,-2631000000.000,636000000.000,...,-8511000000.000,-5799000000.000,655122000.000,62765,7238011,1.500,48970000000,11440000000,43690000000,8450000000
3,11302276702.000,-45143000.000,12018482000.000,-477862000.000,2000000000.000,8.270,146815000.000,12018482000.000,-620704000.000,-424448000.000,...,4744624000.000,2678281000.000,60717000.000,24960,10052271,1.100,11150000000,4960000000,4070000000,7260000000
4,2830067000000.000,-12912000000.000,346747000000.000,18858000000.000,28195000000.000,1.880,5625000000.000,346747000000.000,-35563000000.000,-1445000000.000,...,3240000000.000,56727000000.000,15955718000.000,68254,98734394,1.300,387540000000,170780000000,111110000000,2540000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,7664602549.000,-628000000.000,7643000000.000,-11000000.000,-986000000.000,-4.410,-105000000.000,7643000000.000,-329000000.000,-138000000.000,...,456000000.000,3811000000.000,215500000.000,56466,9251884,0.900,3920000000,2130000000,2170000000,8780000000
571,14000000000.000,-226000000.000,7952000000.000,428000000.000,500000000.000,1.960,85000000.000,7952000000.000,-790000000.000,-191000000.000,...,2292000000.000,3494000000.000,180979000.000,55094,10441670,1.100,5520000000,2090000000,2200000000,18550000000
572,32000000000.000,-369000000.000,5846000000.000,2148000000.000,1281000000.000,4.570,75000000.000,5846000000.000,-1323000000.000,-202000000.000,...,-8507000000.000,-8542000000.000,290000000.000,13082,27578659,1.000,6840000000,3310000000,12660000000,36840000000
573,22002934091.000,-179300000.000,21066000000.000,1241000000.000,222600000.000,1.100,112300000.000,21066000000.000,-1306000000.000,-503600000.000,...,9559300000.000,12020300000.000,210300000.000,63981,14981058,1.000,6940000000,4920000000,6140000000,27560000000


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
train_num=X_train.select_dtypes(include=np.number)
train_cat=X_train.select_dtypes(include=np.object)

test_num=X_test.select_dtypes(include=np.number)
test_cat=X_test.select_dtypes(include=np.object)

transformer = MinMaxScaler().fit(train_num)
train_num_scaled = pd.DataFrame(transformer.transform(train_num), columns=train_num.columns, index=train_num.index)
test_num_scaled = pd.DataFrame(transformer.transform(test_num), columns=test_num.columns, index=test_num.index)

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore').fit(train_cat)
column_name = encoder.get_feature_names_out(train_cat.columns)

train_encoded = pd.DataFrame(encoder.transform(train_cat).toarray(), columns=column_name, index=train_cat.index)
test_encoded = pd.DataFrame(encoder.transform(test_cat).toarray(), columns=column_name, index=test_cat.index)

X_train=pd.concat([train_encoded, train_num_scaled], axis=1)
X_test=pd.concat([test_encoded, test_num_scaled], axis=1)

C:\Users\sabba\AppData\Local\Temp\ipykernel_9636\443914018.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_cat=X_train.select_dtypes(include=np.object)
C:\Users\sabba\AppData\Local\Temp\ipykernel_9636\443914018.py:5: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_cat=X_test.select_dtypes(include=np.object)


In [23]:
def models_automation(models, X_train, y_train,X_test,y_test):
    for model in models:
        model.fit(X_train, y_train)
        print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

In [24]:
model_list = [LinearRegression(),SGDRegressor(),KNeighborsRegressor(), MLPRegressor(),DecisionTreeRegressor(),RandomForestRegressor()]
models_automation(model_list, X_train, y_train,X_test,y_test)

LinearRegression: Train -> 0.8736160520926985, Test -> 0.608695913660819
SGDRegressor: Train -> 0.8668416974931564, Test -> 0.7628893997953599
KNeighborsRegressor: Train -> 0.9177942405559032, Test -> 0.7942954444433303


C:\Users\sabba\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sabba\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sabba\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\sabba\AppData\Local\Temp\ipykernel_9636\2624389277.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using rave

MLPRegressor: Train -> 0.647266413917204, Test -> 0.44610948341570533
DecisionTreeRegressor: Train -> 1.0, Test -> 0.8850834010099693
RandomForestRegressor: Train -> 0.9890119714501416, Test -> 0.9174734899755027


> The model predicted with 91% accuracy the Overall ESG Score 

# Model to predict the ESG Score without the individual ESG Scores

In [42]:
df2=pd.concat([df_num_esg_scores_model, df_cat], axis=1)



X=pd.concat([df2, df_num2], axis=1)
y=labels.drop(['controversey_score'], axis=1)

X

,entity_public_float,comprehensive_loss_net_of_tax,assets,cash_equivalents,comprehensive_income_net_of_tax,earnings_per_share_diluted,income_tax_expense_benefit,liabilities_and_stockholders_equity,net_from_financing_activities,investments,...,median_worker_pay,ceo salary,environment_score,social_score,governance_score,industry,revenue,gross_profit,total_debt,market_cap
0,26600000000.000,-280000000.000,10919000000.000,-431000000.000,1189000000.000,1.190,58000000.000,10919000000.000,-17000000.000,-104000000.000,...,82890,15967631,0.000,9.000,6.000,Health Care,6930000000,3720000000,3130000000,40960000000
1,8200000000.000,-3539000000.000,14756000000.000,505000000.000,930000000.000,-0.680,664000000.000,14756000000.000,-768000000.000,-495000000.000,...,102645,15960369,14.000,6.000,4.000,Materials,12450000000,2310000000,1970000000,7250000000
2,8200000000.000,-4585000000.000,64716000000.000,38000000.000,1484000000.000,0.190,59000000.000,64716000000.000,-2631000000.000,636000000.000,...,62765,7238011,12.000,12.000,5.000,Industrials,48970000000,11440000000,43690000000,8450000000
3,11302276702.000,-45143000.000,12018482000.000,-477862000.000,2000000000.000,8.270,146815000.000,12018482000.000,-620704000.000,-424448000.000,...,24960,10052271,0.100,8.300,3.100,Consumer Discretionary,11150000000,4960000000,4070000000,7260000000
4,2830067000000.000,-12912000000.000,346747000000.000,18858000000.000,28195000000.000,1.880,5625000000.000,346747000000.000,-35563000000.000,-1445000000.000,...,68254,98734394,1.000,7.000,9.000,Information Technology,387540000000,170780000000,111110000000,2540000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,7664602549.000,-628000000.000,7643000000.000,-11000000.000,-986000000.000,-4.410,-105000000.000,7643000000.000,-329000000.000,-138000000.000,...,56466,9251884,2.000,7.000,7.000,Health Care,3920000000,2130000000,2170000000,8780000000
571,14000000000.000,-226000000.000,7952000000.000,428000000.000,500000000.000,1.960,85000000.000,7952000000.000,-790000000.000,-191000000.000,...,55094,10441670,4.000,7.000,5.000,Industrials,5520000000,2090000000,2200000000,18550000000
572,32000000000.000,-369000000.000,5846000000.000,2148000000.000,1281000000.000,4.570,75000000.000,5846000000.000,-1323000000.000,-202000000.000,...,13082,27578659,4.000,12.000,4.000,Consumer Discretionary,6840000000,3310000000,12660000000,36840000000
573,22002934091.000,-179300000.000,21066000000.000,1241000000.000,222600000.000,1.100,112300000.000,21066000000.000,-1306000000.000,-503600000.000,...,63981,14981058,4.000,15.000,8.000,Health Care,6940000000,4920000000,6140000000,27560000000


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
train_num=X_train.select_dtypes(include=np.number)
train_cat=X_train.select_dtypes(include=np.object)

test_num=X_test.select_dtypes(include=np.number)
test_cat=X_test.select_dtypes(include=np.object)

transformer = MinMaxScaler().fit(train_num)
train_num_scaled = pd.DataFrame(transformer.transform(train_num), columns=train_num.columns, index=train_num.index)
test_num_scaled = pd.DataFrame(transformer.transform(test_num), columns=test_num.columns, index=test_num.index)

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore').fit(train_cat)
column_name = encoder.get_feature_names_out(train_cat.columns)

train_encoded = pd.DataFrame(encoder.transform(train_cat).toarray(), columns=column_name, index=train_cat.index)
test_encoded = pd.DataFrame(encoder.transform(test_cat).toarray(), columns=column_name, index=test_cat.index)

X_train=pd.concat([train_encoded, train_num_scaled], axis=1)
X_test=pd.concat([test_encoded, test_num_scaled], axis=1)

C:\Users\sabba\AppData\Local\Temp\ipykernel_19488\1244440316.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_cat=X_train.select_dtypes(include=np.object)
C:\Users\sabba\AppData\Local\Temp\ipykernel_19488\1244440316.py:5: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_cat=X_test.select_dtypes(include=np.object)


In [45]:
X_train=X_train.drop(columns=['environment_score', 'social_score', 'governance_score'])
X_test=X_test.drop(columns=['environment_score', 'social_score', 'governance_score'])

In [46]:
%%time
random = RandomForestRegressor(n_estimators=100,
                             max_depth=100,
                             min_samples_split=3,
                             min_samples_leaf = 3,
                             max_features = 5)
random.fit(X_train, y_train)
y_pred_random = random.predict(X_test)

from sklearn.metrics import mean_squared_error
import math

mse_random = mean_squared_error(y_test, y_pred_random)
 
rmse_random = math.sqrt(mse_random)
print(random.score(X_train, y_train))
print(random.score(X_test, y_test))
print('mse:', mse_random)
print('rmse:', rmse_random)

0.785193838118448
0.28487938362911314
mse: 32.35935659261212
rmse: 5.688528508552288
Wall time: 154 ms


<timed exec>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [47]:
def models_automation(models, X_train, y_train,X_test,y_test):
    for model in models:
        model.fit(X_train, y_train)
        print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

In [48]:
model_list = [LinearRegression(),SGDRegressor(),KNeighborsRegressor(), MLPRegressor(),DecisionTreeRegressor(),RandomForestRegressor()]
models_automation(model_list, X_train, y_train,X_test,y_test)

LinearRegression: Train -> 0.4782438849061107, Test -> -1.7419845646507706
SGDRegressor: Train -> 0.44413014076634105, Test -> 0.2799898939885498
KNeighborsRegressor: Train -> 0.5711221444500243, Test -> 0.2742113583118606


C:\Users\sabba\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sabba\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sabba\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\sabba\AppData\Local\Temp\ipykernel_19488\2624389277.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using rav

MLPRegressor: Train -> 0.41785131831967404, Test -> 0.15367752360655929
DecisionTreeRegressor: Train -> 1.0, Test -> -0.18740746682173803
RandomForestRegressor: Train -> 0.9190568372538466, Test -> 0.29909627878763567


> Former RandonForest result with the categorical column where 0.21. Now with new features are 0.31.

# Model to predict the controversey Score

In [36]:
X=pd.concat([df_num_esg_scores_model, df_cat], axis=1)
y=labels.drop(['esg_score'], axis=1)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
train_num=X_train.select_dtypes(include=np.number)
train_cat=X_train.select_dtypes(include=np.object)

test_num=X_test.select_dtypes(include=np.number)
test_cat=X_test.select_dtypes(include=np.object)

transformer = MinMaxScaler().fit(train_num)
train_num_scaled = pd.DataFrame(transformer.transform(train_num), columns=train_num.columns, index=train_num.index)
test_num_scaled = pd.DataFrame(transformer.transform(test_num), columns=test_num.columns, index=test_num.index)

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore').fit(train_cat)
column_name = encoder.get_feature_names_out(train_cat.columns)

train_encoded = pd.DataFrame(encoder.transform(train_cat).toarray(), columns=column_name, index=train_cat.index)
test_encoded = pd.DataFrame(encoder.transform(test_cat).toarray(), columns=column_name, index=test_cat.index)

X_train=pd.concat([train_encoded, train_num_scaled], axis=1)
X_test=pd.concat([test_encoded, test_num_scaled], axis=1)

C:\Users\sabba\AppData\Local\Temp\ipykernel_9636\1244440316.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_cat=X_train.select_dtypes(include=np.object)
C:\Users\sabba\AppData\Local\Temp\ipykernel_9636\1244440316.py:5: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_cat=X_test.select_dtypes(include=np.object)


In [40]:
X_train=X_train.drop(columns=['environment_score', 'social_score', 'governance_score'])
X_test=X_test.drop(columns=['environment_score', 'social_score', 'governance_score'])

In [41]:
def models_automation(models, X_train, y_train,X_test,y_test):
    for model in models:
        model.fit(X_train, y_train)
        print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

In [42]:
model_list = [LinearRegression(),SGDRegressor(),KNeighborsRegressor(), MLPRegressor(),DecisionTreeRegressor(),RandomForestRegressor()]
models_automation(model_list, X_train, y_train,X_test,y_test)

LinearRegression: Train -> 0.37602422193235896, Test -> 0.19088711758132737
SGDRegressor: Train -> 0.30578007476582647, Test -> 0.26554755562739374
KNeighborsRegressor: Train -> 0.5689723555308769, Test -> 0.12049580324028863


C:\Users\sabba\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sabba\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sabba\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\sabba\AppData\Local\Temp\ipykernel_9636\2624389277.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using rave

MLPRegressor: Train -> 0.41133639317699844, Test -> 0.2412860622693095
DecisionTreeRegressor: Train -> 1.0, Test -> -0.16728479406597696
RandomForestRegressor: Train -> 0.9224147959386031, Test -> 0.3822605406988093


In [43]:
%%time
random = RandomForestRegressor(n_estimators=100,
                             max_depth=100,
                             min_samples_split=3,
                             min_samples_leaf = 3,
                             max_features = 5)
random.fit(X_train, y_train)
y_pred_random = random.predict(X_test)

from sklearn.metrics import mean_squared_error
import math

mse_random = mean_squared_error(y_test, y_pred_random)
 
rmse_random = math.sqrt(mse_random)
print(random.score(X_train, y_train))
print(random.score(X_test, y_test))
print('mse:', mse_random)
print('rmse:', rmse_random)

0.7947069712770034
0.392972389698931
mse: 0.4702914854551798
rmse: 0.6857780147067852
Wall time: 156 ms


<timed exec>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


> Those were the best models results without categoricals:
    
> train score: 0.7896922790617839\
> test score: 0.34766017448305456\
> mse: 0.5440448355254526\
> rmse: 0.7375939503042663

> > Those were the best models results wit categoricals:

> 0.7947069712770034\
> 0.392972389698931\
> mse: 0.4702914854551798\
> rmse: 0.6857780147067852


## Adding feature and running model

In [118]:
df1=pd.read_csv('beta_values2.csv')

In [46]:
X=pd.concat([df_num_esg_scores_model, df_cat, df1], axis=1)
y=labels.drop(['esg_score'], axis=1)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
train_num=X_train.select_dtypes(include=np.number)
train_cat=X_train.select_dtypes(include=np.object)

test_num=X_test.select_dtypes(include=np.number)
test_cat=X_test.select_dtypes(include=np.object)

transformer = MinMaxScaler().fit(train_num)
train_num_scaled = pd.DataFrame(transformer.transform(train_num), columns=train_num.columns, index=train_num.index)
test_num_scaled = pd.DataFrame(transformer.transform(test_num), columns=test_num.columns, index=test_num.index)

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore').fit(train_cat)
column_name = encoder.get_feature_names_out(train_cat.columns)

train_encoded = pd.DataFrame(encoder.transform(train_cat).toarray(), columns=column_name, index=train_cat.index)
test_encoded = pd.DataFrame(encoder.transform(test_cat).toarray(), columns=column_name, index=test_cat.index)

X_train=pd.concat([train_encoded, train_num_scaled], axis=1)
X_test=pd.concat([test_encoded, test_num_scaled], axis=1)

C:\Users\sabba\AppData\Local\Temp\ipykernel_9636\1244440316.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_cat=X_train.select_dtypes(include=np.object)
C:\Users\sabba\AppData\Local\Temp\ipykernel_9636\1244440316.py:5: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_cat=X_test.select_dtypes(include=np.object)


In [49]:
X_train=X_train.drop(columns=['environment_score', 'social_score', 'governance_score'])
X_test=X_test.drop(columns=['environment_score', 'social_score', 'governance_score'])

In [50]:
%%time
random = RandomForestRegressor(n_estimators=100,
                             max_depth=100,
                             min_samples_split=3,
                             min_samples_leaf = 3,
                             max_features = 5)
random.fit(X_train, y_train)
y_pred_random = random.predict(X_test)

from sklearn.metrics import mean_squared_error
import math

mse_random = mean_squared_error(y_test, y_pred_random)
 
rmse_random = math.sqrt(mse_random)
print(random.score(X_train, y_train))
print(random.score(X_test, y_test))
print('mse:', mse_random)
print('rmse:', rmse_random)

0.7939886754833453
0.38659518920840685
mse: 0.47523218838341497
rmse: 0.6893708641822738
Wall time: 160 ms


<timed exec>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [ ]:
# adding more financial data. Total Debt, Total Revenue, Total Debt, Market Cap

In [11]:
df_num1=pd.concat([df_num_controversey_score, df1], axis=1)

In [119]:
df_num2=pd.read_csv('new_features.csv')

In [121]:
df_num2=df_num2.drop(columns='ticker')

In [123]:
df_num_esg_scores_model

,entity_public_float,comprehensive_loss_net_of_tax,assets,cash_equivalents,comprehensive_income_net_of_tax,earnings_per_share_diluted,income_tax_expense_benefit,liabilities_and_stockholders_equity,net_from_financing_activities,investments,net_from_operating_activities,net_income_loss,operating_lease_liability,retained_earnings,stockholders_equity,diluted_shares_outstanding,median_worker_pay,ceo salary
0,26600000000.000,-280000000.000,10919000000.000,-431000000.000,1189000000.000,1.190,58000000.000,10919000000.000,-17000000.000,-104000000.000,296000000.000,352000000.000,159000000.000,541000000.000,5609000000.000,297000000.000,82890,15967631
1,8200000000.000,-3539000000.000,14756000000.000,505000000.000,930000000.000,-0.680,664000000.000,14756000000.000,-768000000.000,-495000000.000,822000000.000,-123000000.000,89000000.000,-570000000.000,5076000000.000,181000000.000,102645,15960369
2,8200000000.000,-4585000000.000,64716000000.000,38000000.000,1484000000.000,0.190,59000000.000,64716000000.000,-2631000000.000,636000000.000,2173000000.000,127000000.000,8024000000.000,-8511000000.000,-5799000000.000,655122000.000,62765,7238011
3,11302276702.000,-45143000.000,12018482000.000,-477862000.000,2000000000.000,8.270,146815000.000,12018482000.000,-620704000.000,-424448000.000,722222000.000,501872000.000,2692861000.000,4744624000.000,2678281000.000,60717000.000,24960,10052271
4,2830067000000.000,-12912000000.000,346747000000.000,18858000000.000,28195000000.000,1.880,5625000000.000,346747000000.000,-35563000000.000,-1445000000.000,34005000000.000,29998000000.000,11470000000.000,3240000000.000,56727000000.000,15955718000.000,68254,98734394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,7664602549.000,-628000000.000,7643000000.000,-11000000.000,-986000000.000,-4.410,-105000000.000,7643000000.000,-329000000.000,-138000000.000,517000000.000,-950000000.000,203000000.000,456000000.000,3811000000.000,215500000.000,56466,9251884
571,14000000000.000,-226000000.000,7952000000.000,428000000.000,500000000.000,1.960,85000000.000,7952000000.000,-790000000.000,-191000000.000,596000000.000,355000000.000,293000000.000,2292000000.000,3494000000.000,180979000.000,55094,10441670
572,32000000000.000,-369000000.000,5846000000.000,2148000000.000,1281000000.000,4.570,75000000.000,5846000000.000,-1323000000.000,-202000000.000,1427000000.000,1325000000.000,707000000.000,-8507000000.000,-8542000000.000,290000000.000,13082,27578659
573,22002934091.000,-179300000.000,21066000000.000,1241000000.000,222600000.000,1.100,112300000.000,21066000000.000,-1306000000.000,-503600000.000,1499200000.000,231400000.000,220300000.000,9559300000.000,12020300000.000,210300000.000,63981,14981058


In [122]:
df_num2

,revenue,gross_profit,total_debt,market_cap
0,6930000000,3720000000,3130000000,40960000000
1,12450000000,2310000000,1970000000,7250000000
2,48970000000,11440000000,43690000000,8450000000
3,11150000000,4960000000,4070000000,7260000000
4,387540000000,170780000000,111110000000,2540000000000
...,...,...,...,...
570,3920000000,2130000000,2170000000,8780000000
571,5520000000,2090000000,2200000000,18550000000
572,6840000000,3310000000,12660000000,36840000000
573,6940000000,4920000000,6140000000,27560000000


In [91]:
df_num

,entity_public_float,comprehensive_loss_net_of_tax,assets,cash_equivalents,comprehensive_income_net_of_tax,earnings_per_share_diluted,income_tax_expense_benefit,liabilities_and_stockholders_equity,net_from_financing_activities,investments,...,retained_earnings,stockholders_equity,diluted_shares_outstanding,median_worker_pay,ceo salary,Beta,revenue,gross_profit,total_debt,market_cap
0,26600000000.000,-280000000.000,10919000000.000,-431000000.000,1189000000.000,1.190,58000000.000,10919000000.000,-17000000.000,-104000000.000,...,541000000.000,5609000000.000,297000000.000,82890,15967631,1.000,6930000000,3720000000,3130000000,40960000000
1,8200000000.000,-3539000000.000,14756000000.000,505000000.000,930000000.000,-0.680,664000000.000,14756000000.000,-768000000.000,-495000000.000,...,-570000000.000,5076000000.000,181000000.000,102645,15960369,2.400,12450000000,2310000000,1970000000,7250000000
2,8200000000.000,-4585000000.000,64716000000.000,38000000.000,1484000000.000,0.190,59000000.000,64716000000.000,-2631000000.000,636000000.000,...,-8511000000.000,-5799000000.000,655122000.000,62765,7238011,1.500,48970000000,11440000000,43690000000,8450000000
3,11302276702.000,-45143000.000,12018482000.000,-477862000.000,2000000000.000,8.270,146815000.000,12018482000.000,-620704000.000,-424448000.000,...,4744624000.000,2678281000.000,60717000.000,24960,10052271,1.100,11150000000,4960000000,4070000000,7260000000
4,2830067000000.000,-12912000000.000,346747000000.000,18858000000.000,28195000000.000,1.880,5625000000.000,346747000000.000,-35563000000.000,-1445000000.000,...,3240000000.000,56727000000.000,15955718000.000,68254,98734394,1.300,387540000000,170780000000,111110000000,2540000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,7664602549.000,-628000000.000,7643000000.000,-11000000.000,-986000000.000,-4.410,-105000000.000,7643000000.000,-329000000.000,-138000000.000,...,456000000.000,3811000000.000,215500000.000,56466,9251884,0.900,3920000000,2130000000,2170000000,8780000000
571,14000000000.000,-226000000.000,7952000000.000,428000000.000,500000000.000,1.960,85000000.000,7952000000.000,-790000000.000,-191000000.000,...,2292000000.000,3494000000.000,180979000.000,55094,10441670,1.100,5520000000,2090000000,2200000000,18550000000
572,32000000000.000,-369000000.000,5846000000.000,2148000000.000,1281000000.000,4.570,75000000.000,5846000000.000,-1323000000.000,-202000000.000,...,-8507000000.000,-8542000000.000,290000000.000,13082,27578659,1.000,6840000000,3310000000,12660000000,36840000000
573,22002934091.000,-179300000.000,21066000000.000,1241000000.000,222600000.000,1.100,112300000.000,21066000000.000,-1306000000.000,-503600000.000,...,9559300000.000,12020300000.000,210300000.000,63981,14981058,1.000,6940000000,4920000000,6140000000,27560000000


In [16]:
df_num=pd.concat([df_num1, df_num2], axis=1)

In [18]:
X=pd.concat([df_num, df_cat], axis=1)
y=labels.drop(['esg_score'], axis=1)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
train_num=X_train.select_dtypes(include=np.number)
train_cat=X_train.select_dtypes(include=np.object)

test_num=X_test.select_dtypes(include=np.number)
test_cat=X_test.select_dtypes(include=np.object)

transformer = MinMaxScaler().fit(train_num)
train_num_scaled = pd.DataFrame(transformer.transform(train_num), columns=train_num.columns, index=train_num.index)
test_num_scaled = pd.DataFrame(transformer.transform(test_num), columns=test_num.columns, index=test_num.index)

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore').fit(train_cat)
column_name = encoder.get_feature_names_out(train_cat.columns)

train_encoded = pd.DataFrame(encoder.transform(train_cat).toarray(), columns=column_name, index=train_cat.index)
test_encoded = pd.DataFrame(encoder.transform(test_cat).toarray(), columns=column_name, index=test_cat.index)

X_train=pd.concat([train_encoded, train_num_scaled], axis=1)
X_test=pd.concat([test_encoded, test_num_scaled], axis=1)

C:\Users\sabba\AppData\Local\Temp\ipykernel_19488\1244440316.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_cat=X_train.select_dtypes(include=np.object)
C:\Users\sabba\AppData\Local\Temp\ipykernel_19488\1244440316.py:5: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_cat=X_test.select_dtypes(include=np.object)


In [22]:
X_train

,industry_Communication Services,industry_Consumer Discretionary,industry_Consumer Staples,industry_Energy,industry_Financials,industry_Health Care,industry_Industrials,industry_Information Technology,industry_Materials,industry_Real Estate,...,retained_earnings,stockholders_equity,diluted_shares_outstanding,median_worker_pay,ceo salary,Beta,revenue,gross_profit,total_debt,market_cap
68,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.070,0.024,0.014,0.067,0.050,0.394,0.160,0.051,0.006,0.006
231,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.072,0.025,0.007,0.084,0.079,0.364,0.096,0.013,0.005,0.005
63,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,...,0.128,0.000,0.037,0.331,0.069,0.364,0.190,0.033,0.091,0.050
436,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,...,0.090,0.031,0.002,0.405,0.027,0.121,0.109,0.011,0.005,0.006
60,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,...,0.066,0.022,0.014,0.160,0.020,0.364,0.098,0.014,0.006,0.003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,...,0.066,0.022,0.007,0.197,0.035,0.515,0.094,0.015,0.013,0.000
106,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.072,0.032,0.009,0.356,0.023,0.303,0.112,0.043,0.005,0.004
270,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,...,0.071,0.027,0.012,0.373,0.094,0.364,0.113,0.030,0.021,0.015
435,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.061,0.024,0.016,0.024,0.051,0.667,0.104,0.018,0.037,0.006


In [23]:
%%time
random = RandomForestRegressor(n_estimators=100,
                             max_depth=100,
                             min_samples_split=3,
                             min_samples_leaf = 3,
                             max_features = 5)
random.fit(X_train, y_train)
y_pred_random = random.predict(X_test)

from sklearn.metrics import mean_squared_error
import math

mse_random = mean_squared_error(y_test, y_pred_random)
 
rmse_random = math.sqrt(mse_random)
print(random.score(X_train, y_train))
print(random.score(X_test, y_test))
print('mse:', mse_random)
print('rmse:', rmse_random)

0.8024241884304475
0.4460070200286317
mse: 0.42920318130711826
rmse: 0.6551360021454463
Wall time: 164 ms


<timed exec>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


> Better improvement:

>Train: 0.8024241884304475\
>Test: 0.4460070200286317\
>mse: 0.42920318130711826\
>rmse: 0.6551360021454463

In [24]:
df

,entity_public_float,comprehensive_loss_net_of_tax,assets,cash_equivalents,comprehensive_income_net_of_tax,earnings_per_share_diluted,income_tax_expense_benefit,liabilities_and_stockholders_equity,net_from_financing_activities,investments,...,diluted_shares_outstanding,ticker,company_name,median_worker_pay,ceo salary,industry,environment_score,social_score,governance_score,controversey_level
0,26600000000.000,-280000000.000,10919000000.000,-431000000.000,1189000000.000,1.190,58000000.000,10919000000.000,-17000000.000,-104000000.000,...,297000000.000,A,"Agilent Technologies, Inc.",82890,15967631,Health Care,0.000,9.000,6.000,Low Controversey
1,8200000000.000,-3539000000.000,14756000000.000,505000000.000,930000000.000,-0.680,664000000.000,14756000000.000,-768000000.000,-495000000.000,...,181000000.000,AA,Alcoa Corporation,102645,15960369,Materials,14.000,6.000,4.000,High Controversey
2,8200000000.000,-4585000000.000,64716000000.000,38000000.000,1484000000.000,0.190,59000000.000,64716000000.000,-2631000000.000,636000000.000,...,655122000.000,AAL,American Airlines Group Inc.,62765,7238011,Industrials,12.000,12.000,5.000,Medium Controversey
3,11302276702.000,-45143000.000,12018482000.000,-477862000.000,2000000000.000,8.270,146815000.000,12018482000.000,-620704000.000,-424448000.000,...,60717000.000,AAP,"Advance Auto Parts, Inc.",24960,10052271,Consumer Discretionary,0.100,8.300,3.100,Medium Controversey
4,2830067000000.000,-12912000000.000,346747000000.000,18858000000.000,28195000000.000,1.880,5625000000.000,346747000000.000,-35563000000.000,-1445000000.000,...,15955718000.000,AAPL,Apple Inc.,68254,98734394,Information Technology,1.000,7.000,9.000,High Controversey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,7664602549.000,-628000000.000,7643000000.000,-11000000.000,-986000000.000,-4.410,-105000000.000,7643000000.000,-329000000.000,-138000000.000,...,215500000.000,XRAY,DENTSPLY SIRONA Inc.,56466,9251884,Health Care,2.000,7.000,7.000,Low Controversey
571,14000000000.000,-226000000.000,7952000000.000,428000000.000,500000000.000,1.960,85000000.000,7952000000.000,-790000000.000,-191000000.000,...,180979000.000,XYL,Xylem Inc.,55094,10441670,Industrials,4.000,7.000,5.000,Low Controversey
572,32000000000.000,-369000000.000,5846000000.000,2148000000.000,1281000000.000,4.570,75000000.000,5846000000.000,-1323000000.000,-202000000.000,...,290000000.000,YUM,"Yum! Brands, Inc.",13082,27578659,Consumer Discretionary,4.000,12.000,4.000,Medium Controversey
573,22002934091.000,-179300000.000,21066000000.000,1241000000.000,222600000.000,1.100,112300000.000,21066000000.000,-1306000000.000,-503600000.000,...,210300000.000,ZBH,"Zimmer Biomet Holdings, Inc.",63981,14981058,Health Care,4.000,15.000,8.000,Medium Controversey


In [29]:
df_tableau=pd.concat([df, df_num2], axis=1)

In [92]:
df_tableau

,entity_public_float,comprehensive_loss_net_of_tax,assets,cash_equivalents,comprehensive_income_net_of_tax,earnings_per_share_diluted,income_tax_expense_benefit,liabilities_and_stockholders_equity,net_from_financing_activities,investments,...,environment_score,social_score,governance_score,esg_score,controversey_score,controversey_level,revenue,gross_profit,total_debt,market_cap
0,26600000000.000,-280000000.000,10919000000.000,-431000000.000,1189000000.000,1.190,58000000.000,10919000000.000,-17000000.000,-104000000.000,...,0.000,9.000,6.000,15.000,1.000,Low Controversey,6930000000,3720000000,3130000000,40960000000
1,8200000000.000,-3539000000.000,14756000000.000,505000000.000,930000000.000,-0.680,664000000.000,14756000000.000,-768000000.000,-495000000.000,...,14.000,6.000,4.000,24.000,3.000,High Controversey,12450000000,2310000000,1970000000,7250000000
2,8200000000.000,-4585000000.000,64716000000.000,38000000.000,1484000000.000,0.190,59000000.000,64716000000.000,-2631000000.000,636000000.000,...,12.000,12.000,5.000,29.000,2.000,Medium Controversey,48970000000,11440000000,43690000000,8450000000
3,11302276702.000,-45143000.000,12018482000.000,-477862000.000,2000000000.000,8.270,146815000.000,12018482000.000,-620704000.000,-424448000.000,...,0.100,8.300,3.100,12.000,2.000,Medium Controversey,11150000000,4960000000,4070000000,7260000000
4,2830067000000.000,-12912000000.000,346747000000.000,18858000000.000,28195000000.000,1.880,5625000000.000,346747000000.000,-35563000000.000,-1445000000.000,...,1.000,7.000,9.000,17.000,3.000,High Controversey,387540000000,170780000000,111110000000,2540000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,7664602549.000,-628000000.000,7643000000.000,-11000000.000,-986000000.000,-4.410,-105000000.000,7643000000.000,-329000000.000,-138000000.000,...,2.000,7.000,7.000,16.000,1.000,Low Controversey,3920000000,2130000000,2170000000,8780000000
571,14000000000.000,-226000000.000,7952000000.000,428000000.000,500000000.000,1.960,85000000.000,7952000000.000,-790000000.000,-191000000.000,...,4.000,7.000,5.000,16.000,1.000,Low Controversey,5520000000,2090000000,2200000000,18550000000
572,32000000000.000,-369000000.000,5846000000.000,2148000000.000,1281000000.000,4.570,75000000.000,5846000000.000,-1323000000.000,-202000000.000,...,4.000,12.000,4.000,21.000,2.000,Medium Controversey,6840000000,3310000000,12660000000,36840000000
573,22002934091.000,-179300000.000,21066000000.000,1241000000.000,222600000.000,1.100,112300000.000,21066000000.000,-1306000000.000,-503600000.000,...,4.000,15.000,8.000,27.000,2.000,Medium Controversey,6940000000,4920000000,6140000000,27560000000


In [32]:
df_tableau.to_csv('tableau_data.csv', index=False)

## Model for Governance

In [135]:
governance_data=pd.concat([df_cat, X], axis=1) 

In [138]:
governance_data.to_csv('models_data.csv', index=False)

In [127]:
X=pd.concat([df_num_esg_scores_model, df_num2], axis=1)
y=labels.governance_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_num=X_train.select_dtypes(include=np.number)
train_cat=X_train.select_dtypes(include=np.object)

test_num=X_test.select_dtypes(include=np.number)
test_cat=X_test.select_dtypes(include=np.object)

transformer = MinMaxScaler().fit(train_num)
train_num_scaled = pd.DataFrame(transformer.transform(train_num), columns=train_num.columns, index=train_num.index)
test_num_scaled = pd.DataFrame(transformer.transform(test_num), columns=test_num.columns, index=test_num.index)

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore').fit(train_cat)
column_name = encoder.get_feature_names_out(train_cat.columns)

train_encoded = pd.DataFrame(encoder.transform(train_cat).toarray(), columns=column_name, index=train_cat.index)
test_encoded = pd.DataFrame(encoder.transform(test_cat).toarray(), columns=column_name, index=test_cat.index)

X_train=pd.concat([train_encoded, train_num_scaled], axis=1)
X_test=pd.concat([test_encoded, test_num_scaled], axis=1)

#X_train=X_train.drop(columns=['environment_score', 'social_score', 'governance_score'])
#X_test=X_test.drop(columns=['environment_score', 'social_score', 'governance_score'])



C:\Users\sabba\AppData\Local\Temp\ipykernel_19488\2909362348.py:7: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_cat=X_train.select_dtypes(include=np.object)
C:\Users\sabba\AppData\Local\Temp\ipykernel_19488\2909362348.py:10: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_cat=X_test.select_dtypes(include=np.object)


In [128]:
%%time
random = RandomForestRegressor(n_estimators=100,
                             max_depth=100,
                             min_samples_split=3,
                             min_samples_leaf = 3,
                             max_features = 5)
random.fit(X_train, y_train)
y_pred_random = random.predict(X_test)

from sklearn.metrics import mean_squared_error
import math

mse_random = mean_squared_error(y_test, y_pred_random)
 
rmse_random = math.sqrt(mse_random)
print(random.score(X_train, y_train))
print(random.score(X_test, y_test))
print('mse:', mse_random)
print('rmse:', rmse_random)

0.7704133776092075
0.33671810181062356
mse: 4.662254352954363
rmse: 2.1592254057773506
Wall time: 219 ms


In [133]:
results = cross_validate(random,X_train, y_train, cv = 5)

results

{'fit_time': array([0.14862108, 0.13223982, 0.1363194 , 0.13287401, 0.14277816]),
 'score_time': array([0.00753284, 0.00682712, 0.00909996, 0.00735569, 0.00849342]),
 'test_score': array([0.21462404, 0.34323449, 0.21419287, 0.10382943, 0.2516647 ])}

##  Model for Environment

In [77]:
y=df.environment_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_num=X_train.select_dtypes(include=np.number)
train_cat=X_train.select_dtypes(include=np.object)

test_num=X_test.select_dtypes(include=np.number)
test_cat=X_test.select_dtypes(include=np.object)

transformer = MinMaxScaler().fit(train_num)
train_num_scaled = pd.DataFrame(transformer.transform(train_num), columns=train_num.columns, index=train_num.index)
test_num_scaled = pd.DataFrame(transformer.transform(test_num), columns=test_num.columns, index=test_num.index)

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore').fit(train_cat)
column_name = encoder.get_feature_names_out(train_cat.columns)

train_encoded = pd.DataFrame(encoder.transform(train_cat).toarray(), columns=column_name, index=train_cat.index)
test_encoded = pd.DataFrame(encoder.transform(test_cat).toarray(), columns=column_name, index=test_cat.index)

X_train=pd.concat([train_encoded, train_num_scaled], axis=1)
X_test=pd.concat([test_encoded, test_num_scaled], axis=1)

X_train=X_train.drop(columns=['environment_score', 'social_score', 'governance_score'])
X_test=X_test.drop(columns=['environment_score', 'social_score', 'governance_score'])

C:\Users\sabba\AppData\Local\Temp\ipykernel_19488\3606883232.py:6: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_cat=X_train.select_dtypes(include=np.object)
C:\Users\sabba\AppData\Local\Temp\ipykernel_19488\3606883232.py:9: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_cat=X_test.select_dtypes(include=np.object)


In [78]:
%%time
random = RandomForestRegressor(n_estimators=100,
                             max_depth=100,
                             min_samples_split=3,
                             min_samples_leaf = 3,
                             max_features = 5)
random.fit(X_train, y_train)
y_pred_random = random.predict(X_test)

from sklearn.metrics import mean_squared_error
import math

mse_random = mean_squared_error(y_test, y_pred_random)
 
rmse_random = math.sqrt(mse_random)
print(random.score(X_train, y_train))
print(random.score(X_test, y_test))
print('mse:', mse_random)
print('rmse:', rmse_random)

0.840530249972095
0.43697964821879254
mse: 13.500336569084523
rmse: 3.6742804151404287
Wall time: 155 ms


In [79]:
results = cross_validate(random,X_train, y_train, cv = 5)

results

{'fit_time': array([0.12017512, 0.10828209, 0.12361097, 0.10953045, 0.11053252]),
 'score_time': array([0.00683951, 0.0065093 , 0.00600243, 0.0070014 , 0.00600123]),
 'test_score': array([0.54065625, 0.58576301, 0.40003111, 0.46989559, 0.4671973 ])}

## Model for Social

In [85]:
y=df.social_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_num=X_train.select_dtypes(include=np.number)
train_cat=X_train.select_dtypes(include=np.object)

test_num=X_test.select_dtypes(include=np.number)
test_cat=X_test.select_dtypes(include=np.object)

transformer = MinMaxScaler().fit(train_num)
train_num_scaled = pd.DataFrame(transformer.transform(train_num), columns=train_num.columns, index=train_num.index)
test_num_scaled = pd.DataFrame(transformer.transform(test_num), columns=test_num.columns, index=test_num.index)

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore').fit(train_cat)
column_name = encoder.get_feature_names_out(train_cat.columns)

train_encoded = pd.DataFrame(encoder.transform(train_cat).toarray(), columns=column_name, index=train_cat.index)
test_encoded = pd.DataFrame(encoder.transform(test_cat).toarray(), columns=column_name, index=test_cat.index)

X_train=pd.concat([train_encoded, train_num_scaled], axis=1)
X_test=pd.concat([test_encoded, test_num_scaled], axis=1)

X_train=X_train.drop(columns=['environment_score', 'social_score', 'governance_score'])
X_test=X_test.drop(columns=['environment_score', 'social_score', 'governance_score'])

C:\Users\sabba\AppData\Local\Temp\ipykernel_19488\4261304963.py:6: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_cat=X_train.select_dtypes(include=np.object)
C:\Users\sabba\AppData\Local\Temp\ipykernel_19488\4261304963.py:9: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_cat=X_test.select_dtypes(include=np.object)


In [86]:
%%time
random = RandomForestRegressor(n_estimators=100,
                             max_depth=100,
                             min_samples_split=3,
                             min_samples_leaf = 3,
                             max_features = 5)
random.fit(X_train, y_train)
y_pred_random = random.predict(X_test)

from sklearn.metrics import mean_squared_error
import math

mse_random = mean_squared_error(y_test, y_pred_random)
 
rmse_random = math.sqrt(mse_random)
print(random.score(X_train, y_train))
print(random.score(X_test, y_test))
print('mse:', mse_random)
print('rmse:', rmse_random)

0.7476056035338657
0.1699151596931162
mse: 11.675024964366305
rmse: 3.416873565756612
Wall time: 155 ms


In [73]:
from sklearn.model_selection import cross_validate

In [87]:
results = cross_validate(random,X_train, y_train, cv = 5)

results

{'fit_time': array([0.11319923, 0.11015606, 0.10987234, 0.11422944, 0.11102438]),
 'score_time': array([0.0070219 , 0.00655198, 0.00700164, 0.00600123, 0.00600123]),
 'test_score': array([0.2356597 , 0.23270162, 0.15253317, 0.30468734, 0.09351515])}

In [94]:
df_num.to_csv('model_data_num.csv', index=False)

In [129]:
from pickle import dump

In [ ]:
# Saving the model

In [130]:
dump(random, open('model.pkl', 'wb'))

In [ ]:
# Saving the Scaler

In [131]:
dump(transformer, open('scaler.pkl', 'wb'))

In [ ]:
# Saving the Encoder

In [132]:
dump(encoder, open('encoder.pkl', 'wb'))

In [103]:
X_train

,industry_Communication Services,industry_Consumer Discretionary,industry_Consumer Staples,industry_Energy,industry_Financials,industry_Health Care,industry_Industrials,industry_Information Technology,industry_Materials,industry_Real Estate,...,operating_lease_liability,retained_earnings,stockholders_equity,diluted_shares_outstanding,median_worker_pay,ceo salary,revenue,gross_profit,total_debt,market_cap
68,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.041,0.070,0.024,0.014,0.067,0.050,0.160,0.051,0.006,0.006
231,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.005,0.072,0.025,0.007,0.084,0.079,0.096,0.013,0.005,0.005
63,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,...,0.023,0.128,0.000,0.037,0.331,0.069,0.190,0.033,0.091,0.050
436,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,...,0.002,0.090,0.031,0.002,0.405,0.027,0.109,0.011,0.005,0.006
60,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,...,0.002,0.066,0.022,0.014,0.160,0.020,0.098,0.014,0.006,0.003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,...,0.001,0.066,0.022,0.007,0.197,0.035,0.094,0.015,0.013,0.000
106,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.002,0.072,0.032,0.009,0.356,0.023,0.112,0.043,0.005,0.004
270,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,...,0.005,0.071,0.027,0.012,0.373,0.094,0.113,0.030,0.021,0.015
435,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.009,0.061,0.024,0.016,0.024,0.051,0.104,0.018,0.037,0.006


In [104]:
X_train.columns


Index(['industry_Communication Services', 'industry_Consumer Discretionary',
       'industry_Consumer Staples', 'industry_Energy', 'industry_Financials',
       'industry_Health Care', 'industry_Industrials',
       'industry_Information Technology', 'industry_Materials',
       'industry_Real Estate', 'industry_Utilities', 'entity_public_float',
       'comprehensive_loss_net_of_tax', 'assets', 'cash_equivalents',
       'comprehensive_income_net_of_tax', 'earnings_per_share_diluted',
       'income_tax_expense_benefit', 'liabilities_and_stockholders_equity',
       'net_from_financing_activities', 'investments',
       'net_from_operating_activities', 'net_income_loss',
       'operating_lease_liability', 'retained_earnings', 'stockholders_equity',
       'diluted_shares_outstanding', 'median_worker_pay', 'ceo salary',
       'revenue', 'gross_profit', 'total_debt', 'market_cap'],
      dtype='object')

In [110]:
train_num.columns

Index(['entity_public_float', 'comprehensive_loss_net_of_tax', 'assets',
       'cash_equivalents', 'comprehensive_income_net_of_tax',
       'earnings_per_share_diluted', 'income_tax_expense_benefit',
       'liabilities_and_stockholders_equity', 'net_from_financing_activities',
       'investments', 'net_from_operating_activities', 'net_income_loss',
       'operating_lease_liability', 'retained_earnings', 'stockholders_equity',
       'diluted_shares_outstanding', 'median_worker_pay', 'ceo salary',
       'environment_score', 'social_score', 'governance_score', 'revenue',
       'gross_profit', 'total_debt', 'market_cap'],
      dtype='object')